In [ ]:
!pip install gradio

In [ ]:
#  cd /tmp && npm install localtunnel && npx localtunnel --port 8501 -y
# import urllib
# print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [6]:
import gradio as gr
from PIL import Image
import requests
from io import BytesIO
import hopsworks
import torch
from transformers import CLIPProcessor, CLIPModel
import functools


def get_image_embedding(image: Image.Image, processor: CLIPProcessor, model: CLIPModel, device: str):
    try:
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        # L2-normalize
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        return image_features.squeeze().cpu().tolist()
    except Exception as e:
        print(f"Error computing embedding: {e}")
        raise


# Hopsworks setup
proj = hopsworks.login()

fs = proj.get_feature_store()
fg = fs.get_feature_group("image_embeddings", version=1)
mr = proj.get_model_registry()
model_mr = mr.get_model("openaiclip_vit_base_patch32", version=1)
dir = model_mr.download()

# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(dir).to(device).eval()
processor = CLIPProcessor.from_pretrained(dir)


def load_image_from_url(url, model, processor, device, fg):
    try:
        # load query image
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))

        # get embedding and neighbors
        embedding = get_image_embedding(image, processor, model, device)
        results = fg.find_neighbors(embedding, k=3)

        returned_files = []
        returned_images = []
        for result in results:
            path = result[1][0]  # adapt to actual structure
            returned_files.append(path)
            try:
                img = Image.open(path)
                returned_images.append(img)
            except Exception as e:
                print(f"Error opening {path}: {e}")
                returned_images.append(None)

        return (
            image,  # main image
            f"Format: {image.format}, Size: {image.size}, Mode: {image.mode}",
            returned_images,
            "\n".join(returned_files)
        )

    except Exception as e:
        return None, f"Error: {str(e)}", [], ""


# wrap function
load_image = functools.partial(load_image_from_url, model=model, processor=processor, device=device, fg=fg)


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Load Image from URL and Find Similar Images")

    with gr.Row():
        url_input = gr.Textbox(label="Enter image URL")
        output_image = gr.Image(type="pil", label="Input Image")
        output_info = gr.Textbox(label="Image details")

    with gr.Row():
        similar_images = gr.Gallery(label="Similar Images", columns=3, rows=1)

    with gr.Row():
        similar_files = gr.Textbox(label="Similar Image Filenames")

    url_input.change(
        load_image,
        inputs=[url_input],
        outputs=[output_image, output_info, similar_images, similar_files]
    )

demo.launch(share=True)


Connection closed.
2025-08-25 15:23:08,353 INFO: Python Engine initialized.

Logged in to project, explore it here https://snurran.devnet.hops.works/p/120
* Running on local URL:  http://127.0.0.1:7864. DONE
2025-08-25 15:23:12,388 INFO: HTTP Request: GET http://127.0.0.1:7864/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-08-25 15:23:12,393 INFO: HTTP Request: HEAD http://127.0.0.1:7864/ "HTTP/1.1 200 OK"
2025-08-25 15:23:12,851 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-08-25 15:23:13,040 INFO: HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"
* Running on public URL: https://ebc3d8ae4f3ccead9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
2025-08-25 15:23:14,895 INFO: HTTP Request: HEAD https://ebc3d8ae4f3ccead9e.gradio.live "HTTP/1.1 200 OK"


2025-08-25 15:23:26,318 WARNING: POST https://rest.elastic.service.consul:9200/120__embedding_default_project_embedding_0/_search?timeout=30s [status:N/A request:0.000s]
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/urllib3/connectionpool.py", line 276, in _get_conn
    conn = self.pool.get(block=self.block, timeout=timeout)
AttributeError: 'NoneType' object has no attribute 'get'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py", line 249, in perform_request
    response = self.pool.urlopen(
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/urllib3/connectionpool.py", line 705, in urlopen
    conn = self._get_conn(timeout=pool_timeout)
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/si